# Regression: Linear

## References

https://www.kaggle.com/datasets/abrambeyer/openintro-possum

## Import Python Libraries

In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif, RFE, SelectFromModel
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler, MinMaxScaler

## Global Variables

## Import Dataset

In [ ]:

# Source: https://www.kaggle.com/datasets/abrambeyer/openintro-possum

# Dataset Column Overview: 
# ------------------------------------------------------------------------------

# X01: case: observation number
# X02: site: The site number where the possum was trapped.
# X03: Pop: Population, either Vic (Victoria) or other (New South Wales or Queensland).
# X04: sex: Gender, either m (male) or f (female).
#  y1: age: Age.
# X05: hdlngth: Head length, in mm.
# X06: skullw: Skull width, in mm.
# X07: totlngth: Total length, in cm.
# X08: taill: Tail length, in cm.
# X09: footlgth: foot length
# X10: earconch: ear conch length
# X11: eye: distance from medial canthus to lateral canthus of right eye
# X11: chest: chest girth (in cm)
# X12: belly: belly girth (in cm)

In [63]:
# load dataset from github
filepath1 = 'https://raw.githubusercontent.com/notfakearcher/julian/main/02_data/possum.csv'
df0 = pd.read_csv(filepath1, header = 0)
df0.head()

,Gender,Height,Weight,Index
0,Male,174,96,4
1,Male,189,87,2
2,Female,185,110,4
3,Female,195,104,3
4,Male,149,61,3
